In [1]:
! pip install pyarrow pandas

data_path = "./sst-2/"
data_list = [
    "test-00000-of-00001.parquet",
    "train-00000-of-00001.parquet",
    "validation-00000-of-00001.parquet"
]
import pyarrow.parquet as pq
import os
# 读取 Parquet 文件
table = pq.read_table(os.path.join(data_path,data_list[1]))

# 转换为 Pandas DataFrame
df = table.to_pandas()
df
len(df)


sentence = df["sentence"]
print(sentence)
print(type(sentence))
sentence[0]


Looking in indexes: http://mirrors.aliyun.com/pypi/simple
0             hide new secretions from the parental units 
1                     contains no wit , only labored gags 
2        that loves its characters and communicates som...
3        remains utterly satisfied to remain the same t...
4        on the worst revenge-of-the-nerds clichés the ...
                               ...                        
67344                                 a delightful comedy 
67345                     anguish , anger and frustration 
67346    at achieving the modest , crowd-pleasing goals...
67347                                    a patient viewer 
67348    this new jangle of noise , mayhem and stupidit...
Name: sentence, Length: 67349, dtype: object
<class 'pandas.core.series.Series'>


'hide new secretions from the parental units '

In [2]:
# 选n个样本做demonstration （from train set）
# 用全部validation set做query
# 得到一个acc
# 这就给这n个sample一个score了
# 然后每一个sample出现m次
# 最后的score就是m次的均值


# 查看数据集的前几行
print(df.head())

# 查看数据集的描述性统计信息
print(df.describe())

# 查看数据集的列名
print(df.columns)

   idx                                           sentence  label
0    0       hide new secretions from the parental units       0
1    1               contains no wit , only labored gags       0
2    2  that loves its characters and communicates som...      1
3    3  remains utterly satisfied to remain the same t...      0
4    4  on the worst revenge-of-the-nerds clichés the ...      0
                idx         label
count  67349.000000  67349.000000
mean   33674.000000      0.557826
std    19442.125977      0.496649
min        0.000000      0.000000
25%    16837.000000      0.000000
50%    33674.000000      1.000000
75%    50511.000000      1.000000
max    67348.000000      1.000000
Index(['idx', 'sentence', 'label'], dtype='object')


In [ ]:
sentence = df["sentence"]
print(sentence)
print(type(sentence))
sentence[0]

In [48]:
# define  m = 5  (frequence)   n = 5  (demonstrations)
# 选择可以用两个list，第一个是0-len(trainset) 第二个是全设为5，每次选择5个，选完之后，删除，然后是一次中，每个实例只能选一次, random_seed保证实验可重复性
import random
random.seed(32)
def sample(m,n,train_set):
    # 裁减 m*len(train_set)//n
    if m*len(train_set)%n != 0 :
        train_set = train_set[:len(train_set)-(m*len(train_set))%n]
    length = len(train_set)
    num_list = [m]*length # 计数
    count_list = [i for i in range(length)] # 采样用
    epoch = m*len(train_set)//n

    result = []
    
    for _ in range(epoch):
        ls = []
        for _ in range(n):

            random_num = random.randint(0,len(count_list)-1) # 随机采样，采样下标

            # print(random_num,len(count_list))

            while count_list[random_num] in ls:  # 取消重复
                random_num = random.randint(0,len(count_list)-1)
                # print(random_num,len(count_list))
            ls.append(count_list[random_num]) # 添加内容
            num_list[random_num] -= 1
            # print(num_list)
            if num_list[random_num] <= 0 : # 次数够就删掉
                num_list.pop(random_num)
                count_list.pop(random_num)
        result.append(ls)
        # print(ls)
    return result
trainset = [0 for i in range(1000) ]
nested_list = sample(3,3,trainset)
# nested_list


In [ ]:
def validate_and_count(nested_list):
    # 扁平化列表
    flat_list = [item for sublist in nested_list for item in sublist]
    
    # 去重并排序
    unique_sorted_list = sorted(set(flat_list))
    
    # 计算每个元素出现的次数
    count_list = [flat_list.count(num) for num in unique_sorted_list]
    
    return unique_sorted_list, count_list

# 示例输入

unique_sorted, counts = validate_and_count(nested_list)

# 输出结果
print("Unique Sorted List:", unique_sorted)
print("Counts List:", counts)

In [ ]:
num_list = [5]*5
num_list
count_list = [i for i in range(5)]
count_list
import random
random.randint(0,10)

In [ ]:
import random

def sample(m, n, train_set):
    # 裁减 m*len(train_set)//n
    if m * len(train_set) % n != 0:
        train_set = train_set[:len(train_set) - (m * len(train_set)) % n]
    length = len(train_set)
    num_list = [m] * length  # 计数
    count_list = list(range(length))  # 采样用
    epoch = m * len(train_set) // n
    result = []
    
    for _ in range(epoch):
        ls = []
        for _ in range(n):
            if not count_list:
                break  # 如果没有可采样的元素，提前退出

            random_num = random.randint(0, len(count_list) - 1)  # 修正随机数范围
            while count_list[random_num] in ls:  # 取消重复
                random_num = random.randint(0, len(count_list) - 1)

            ls.append(count_list[random_num])  # 添加内容
            num_list[count_list[random_num]] -= 1
            
            if num_list[count_list[random_num]] <= 0:  # 次数够就删掉
                num_list[random_num] = num_list[-1]  # 将次数设为0以避免误删
                num_list.pop()
                count_list[random_num] = count_list[-1]  # 将最后一个元素移动到当前索引
                count_list.pop()  # 删除最后一个元素
        result.append(ls)
    
    return result
trainset = [0 for i in range(6) ]
sample(3,3,trainset)
